In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm_notebook
from itertools import product
from typing import Union

import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

import datetime
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [20]:
df = pd.read_csv('FinBERT\Data_final\BTC_final.csv')
df=df.drop('Unnamed: 0',axis=1)
df

,CoinScore,CoinDate,Open,High,Low,Close,Volume
0,-0.224409,2014-10-01,387.427002,391.378998,380.779999,383.614990,2.622940e+07
1,0.162439,2014-10-02,383.988007,385.497009,372.946014,375.071991,2.177770e+07
2,-0.136249,2014-10-03,375.181000,377.695007,357.859009,359.511993,3.090120e+07
3,-0.134887,2014-10-04,359.891998,364.487000,325.885986,328.865997,4.723650e+07
4,-0.134887,2014-10-05,328.915985,341.800995,289.295990,320.510010,8.330810e+07
...,...,...,...,...,...,...,...
3068,-0.530137,2023-02-24,23946.007810,24103.705080,23007.072270,23198.126950,2.681174e+10
3069,-0.524836,2023-02-25,23200.125000,23210.210940,22861.558590,23175.375000,1.610072e+10
3070,-0.524836,2023-02-26,23174.150390,23654.367190,23084.220700,23561.212890,1.664453e+10
3071,-0.122506,2023-02-27,23561.451170,23857.890630,23205.878910,23522.871090,2.266076e+10


In [21]:
#將時間轉換為數值，以方便後續可以運算
timestamp_s = pd.to_datetime(df['CoinDate']).map(datetime.datetime.timestamp)
df['Coin_timestamp'] = timestamp_s
df = df.drop(['CoinDate'], axis=1)
df

,CoinScore,Open,High,Low,Close,Volume,Coin_timestamp
0,-0.224409,387.427002,391.378998,380.779999,383.614990,2.622940e+07,1.412093e+09
1,0.162439,383.988007,385.497009,372.946014,375.071991,2.177770e+07,1.412179e+09
2,-0.136249,375.181000,377.695007,357.859009,359.511993,3.090120e+07,1.412266e+09
3,-0.134887,359.891998,364.487000,325.885986,328.865997,4.723650e+07,1.412352e+09
4,-0.134887,328.915985,341.800995,289.295990,320.510010,8.330810e+07,1.412438e+09
...,...,...,...,...,...,...,...
3068,-0.530137,23946.007810,24103.705080,23007.072270,23198.126950,2.681174e+10,1.677168e+09
3069,-0.524836,23200.125000,23210.210940,22861.558590,23175.375000,1.610072e+10,1.677254e+09
3070,-0.524836,23174.150390,23654.367190,23084.220700,23561.212890,1.664453e+10,1.677341e+09
3071,-0.122506,23561.451170,23857.890630,23205.878910,23522.871090,2.266076e+10,1.677427e+09


In [22]:
target = df['Close']
exog = df[['Open', 'High', 'Low', 'Volume', 'Coin_timestamp','CoinScore']]

In [23]:
ad_fuller_result = adfuller(target)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

ADF Statistic: -1.6052963344633844
p-value: 0.4809324510344754


In [24]:
target_diff = target.diff()

ad_fuller_result = adfuller(target_diff[1:])

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

ADF Statistic: -9.282630023506483
p-value: 1.2393192939570603e-15


In [25]:
def optimize_SARIMAX(endog: Union[pd.Series, list], exog: Union[pd.Series, list], order_list: list, d: int, D: int, s: int) -> pd.DataFrame:
    
    results = []
    
    for order in tqdm_notebook(order_list):
        try: 
            model = SARIMAX(
                endog,
                exog,
                order=(order[0], d, order[1]),
                seasonal_order=(order[2], D, order[3], s),
                simple_differencing=False).fit(disp=False)
        except:
            continue
            
        aic = model.aic
        results.append([order, aic])
        
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q,P,Q)', 'AIC']
    
    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
    
    return result_df

In [26]:
p = range(0, 4, 1)
d = 1
q = range(0, 4, 1)
P = range(0, 1, 1)
D = 0
Q = range(0, 1, 1)
s = 7

parameters = product(p, q, P, Q)
parameters_list = list(parameters)

In [27]:
target_train = target[:2151]
exog_train = exog[:2151]

result_df = optimize_SARIMAX(target_train, exog_train, parameters_list, d, D, s)
result_df

  0%|          | 0/16 [00:00<?, ?it/s]

,"(p,q,P,Q)",AIC
0,"(3, 3, 0, 0)",26995.060269
1,"(1, 2, 0, 0)",27024.572053
2,"(0, 2, 0, 0)",27028.668561
3,"(2, 0, 0, 0)",27030.052425
4,"(1, 1, 0, 0)",27030.081090
5,"(3, 0, 0, 0)",27030.830850
6,"(0, 3, 0, 0)",27031.342276
7,"(1, 3, 0, 0)",27031.793016
8,"(2, 1, 0, 0)",27032.980719
9,"(3, 1, 0, 0)",27034.604974


In [28]:
best_model = SARIMAX(target_train, exog_train, order=(3,1,3), seasonal_order=(0,0,0,7), simple_differencing=False)
best_model_fit = best_model.fit(disp=False)

print(best_model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 2151
Model:               SARIMAX(3, 1, 3)   Log Likelihood              -13484.530
Date:                Sun, 30 Apr 2023   AIC                          26995.060
Time:                        01:44:59   BIC                          27068.812
Sample:                             0   HQIC                         27022.041
                               - 2151                                         
Covariance Type:                  opg                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Open              -0.6050      0.004   -142.394      0.000      -0.613      -0.597
High               0.6945      0.005    135.740      0.000       0.684       0.705
Low                0.6169      0.005

In [29]:
def recursive_forecast(endog: Union[pd.Series, list], exog: Union[pd.Series, list], train_len: int, horizon: int, window: int, method: str) -> list:
    
    total_len = train_len + horizon

    if method == 'last':
        pred_last_value = []
        
        for i in range(train_len, total_len, window):
            last_value = endog[:i].iloc[-1]
            pred_last_value.extend(last_value for _ in range(window))
            
        return pred_last_value
    
    elif method == 'SARIMAX':
        pred_SARIMAX = []
        
        for i in range(train_len, total_len, window):
            model = SARIMAX(endog[:i], exog[:i], order=(3,1,3), seasonal_order=(0,0,0,7), simple_differencing=False)
            res = model.fit(disp=False)
            predictions = res.get_prediction(exog=exog)
            oos_pred = predictions.predicted_mean.iloc[-window:]
            pred_SARIMAX.extend(oos_pred)
            
        return pred_SARIMAX

In [30]:
target_train = target[:3066]
target_test = target[3066:]

pred_df = pd.DataFrame({'actual': target_test})

TRAIN_LEN = len(target_train)
HORIZON = len(target_test)
WINDOW = 1

pred_SARIMAX = recursive_forecast(target, exog, TRAIN_LEN, HORIZON, WINDOW, 'SARIMAX')

pred_df['pred_SARIMAX'] = pred_SARIMAX

pred_df

,actual,pred_SARIMAX
3066,24188.84375,24768.773977
3067,23947.49219,23891.057892
3068,23198.12695,24443.585731
3069,23175.37500,23356.189465
3070,23561.21289,22893.682834
3071,23522.87109,23640.721608
3072,23433.81641,23565.520908


In [31]:
mean_absolute_error(pred_df.actual, pred_df.pred_SARIMAX)

425.67469182453505